https://mp.weixin.qq.com/s/UFiZsPfXwg8lQUsGXcgHjA

In [1]:
import pandas as pd
import numpy as np
import xgboost as xgb
from tqdm import tqdm
from sklearn.svm import SVC
from keras.models import Sequential
from keras.layers.recurrent import LSTM, GRU
from keras.layers.core import Dense, Activation, Dropout
from keras.layers.embeddings import Embedding
from keras.layers.normalization import BatchNormalization
from keras.utils import np_utils
from sklearn import preprocessing, decomposition, model_selection, metrics, pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from keras.layers import GlobalMaxPooling1D, Conv1D, MaxPooling1D, Flatten, Bidirectional, SpatialDropout1D
from keras.preprocessing import sequence, text
from keras.callbacks import EarlyStopping
from nltk import word_tokenize

C:\Users\jianghaitao1\AppData\Local\Continuum\anaconda3\lib\site-packages\numpy\_distributor_init.py:32: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\jianghaitao1\AppData\Local\Continuum\anaconda3\lib\site-packages\numpy\.libs\libopenblas.IPBC74C7KURV7CB2PKT5Z5FNR3SIBV4J.gfortran-win_amd64.dll
C:\Users\jianghaitao1\AppData\Local\Continuum\anaconda3\lib\site-packages\numpy\.libs\libopenblas.NOIJJG62EMASZI6NYURL6JBKM4EVBGM7.gfortran-win_amd64.dll
C:\Users\jianghaitao1\AppData\Local\Continuum\anaconda3\lib\site-packages\numpy\.libs\libopenblas.txa6yqsd3gcqqc22geq54j2udcxdxhwn.gfortran-win_amd64.dll
  stacklevel=1)
Using TensorFlow backend.
C:\Users\jianghaitao1\AppData\Local\Continuum\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:523: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\jiangha

In [3]:
data = pd.read_excel('../../dataset/复旦大学中文文本分类语料.xlsx','sheet1')
data.head()

,分类,正文
0,艺术,﻿【 文献号 】1-2432\n【原文出处】出版发行研究\n【原刊地名】京\n【原刊期号】1...
1,艺术,﻿【 文献号 】1-2435\n【原文出处】扬州师院学报：社科版\n【原刊期号】199504...
2,艺术,﻿【 文献号 】1-2785\n【原文出处】南通师专学报：社科版\n【原刊期号】199503...
3,艺术,﻿【 文献号 】1-3021\n【原文出处】社会科学战线\n【原刊地名】长春\n【原刊期号】...
4,艺术,﻿【 文献号 】1-3062\n【原文出处】上海文化\n【原刊期号】199505\n【原刊页...


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9249 entries, 0 to 9248
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   分类      9249 non-null   object
 1   正文      9249 non-null   object
dtypes: object(2)
memory usage: 144.6+ KB


In [5]:
data.iloc[0,1]

'\ufeff【 文献号 】1-2432\n【原文出处】出版发行研究\n【原刊地名】京\n【原刊期号】199601\n【原刊页号】40-41\n【分 类 号】Z1\n【分 类 名】出版工作、图书评介\n【 作者 】王益\n【复印期号】199604\n【 标题 】美国出版社怎样经营管理？——介绍《图书出版的艺术和科学》\n【 正文 】\n美国出版的有关图书出版的专业书籍，概论性的有好几本，专讲经营管理的并不多，《图书出版的艺术和科学》（The Art andScienceo\nf Book Pblishing）是经常被人推荐的一本。作者小赫伯特·S·贝利（Herbert·S·Bailey,Jr.），在大学文学系毕业之后，1946年进入普林\n斯顿大学出版社当了8年编辑，1954年出任该出版社社长， 直至1986年退休，从事出版工作共40年。1970年，他写了这本书，1980年再版，199\n0年三版。这本书，广泛地被大学采用作出版课程教材，也被出版社工作人员选作参考读物。从1970年至1990年，时隔20年，他认为他所阐述的\n基本原则仍是正确的，因而出版时基本上没有修改。这本书是作者长期实践经验的总结，他也从管理科学和财会科学书籍中吸取了营养，并听取\n了其他出版家的意见，它是写给出版社的社长们读的，也是写给出版社所有的工作人员读的。有关出版工作的方方面面，它都涉及到了。他对各\n项工作的甜酸苦辣好像都有切身的体会，他了解其中的主要矛盾和麻烦，并对如何解决这些问题提出了很好的建议。他看问题全面、客观，立论\n公正，处处迸发出智慧的火花。凡读过这本书的人，都会感到得益。美国《出版商周刊》曾在书评中把它誉为“出版业经营管理方面不可缺少的\n有说服力的研究著作”。\n本书从论述出版工作中的理性和非理性开始，作者认为，出版社的经营管理者是一个有理性的人，在理性的环境中与有理性的人们一道工作\n，追求可能是复杂的但至少可以明确表示的目标，而整个出版活动，又沉浸在非理性的大海中。经营管理必须把非理性因素也考虑进去，而不能\n企图将其纳入既定秩序的框架之内，对非理性因素，要在内部和外部和它生活、工作在一起，参与进去，并且试图理解它，甚至促进它、鼓励它\n、批评它，把它和出版社的健康现象联系起来。作者认为，非理性也是一种重要的动力，许多培育出版业成长壮大的创造力来自非理性，来自作\

In [6]:
import jieba
# jieba.enable_parallel(4) # 并行分词开启
data['文本分词'] = data['正文'].apply(lambda i:jieba.cut(i) )
data['文本分词'] =[' '.join(i) for i in data['文本分词']]

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\JIANGH~1\AppData\Local\Temp\jieba.cache
Loading model cost 0.583 seconds.
Prefix dict has been built succesfully.


In [10]:
pd.set_option('display.max_columns',10000)
pd.set_option('display.width',100000)
pd.set_option('display.max_rows',10000)
pd.set_option('max_colwidth',10000)

In [11]:
data['文本分词'][0:1]

0    ﻿ 【   文献号   】 1 - 2432 \n 【 原文 出处 】 出版发行 研究 \n 【 原刊 地名 】 京 \n 【 原刊 期号 】 199601 \n 【 原刊 页 号 】 40 - 41 \n 【 分   类   号 】 Z1 \n 【 分   类   名 】 出版 工作 、 图书 评介 \n 【   作者   】 王益 \n 【 复印 期号 】 199604 \n 【   标题   】 美国 出版社 怎样 经营 管理 ？ — — 介绍 《 图书 出版 的 艺术 和 科学 》 \n 【   正文   】 \n 美国 出版 的 有关 图书 出版 的 专业书籍 ， 概论 性 的 有 好几本 ， 专讲 经营 管理 的 并不多 ， 《 图书 出版 的 艺术 和 科学 》 （ The   Art   andScienceo \n f   Book   Pblishing ） 是 经常 被 人 推荐 的 一本 。 作者 小 赫伯特 · S · 贝利 （ Herbert · S · Bailey , Jr . ） ， 在 大学 文学系 毕业 之后 ， 1946 年 进入 普林 \n 斯顿 大学 出版社 当 了 8 年 编辑 ， 1954 年 出任 该 出版社 社长 ，   直至 1986 年 退休 ， 从事 出版 工作 共 40 年 。 1970 年 ， 他 写 了 这 本书 ， 1980 年 再版 ， 199 \n 0 年三版 。 这 本书 ， 广泛 地被 大学 采用 作 出版 课程 教材 ， 也 被 出版社 工作人员 选作 参考 读物 。 从 1970 年 至 1990 年 ， 时隔 20 年 ， 他 认为 他 所 阐述 的 \n 基本 原则 仍 是 正确 的 ， 因而 出版 时 基本上 没有 修改 。 这 本书 是 作者 长期实践 经验 的 总结 ， 他 也 从 管理科学 和 财会 科学 书籍 中 吸取 了 营养 ， 并 听取 \n 了 其他 出版家 的 意见 ， 它 是 写给 出版社 的 社长 们 读 的 ， 也 是 写给 出版社 所有 的 工作人员 读 的 。 有关 出版 工作 的 方方面面 ， 它 都 涉及 到 了 。 他 对 各 \n 项 工作 的 甜酸苦辣 好像 都 有 切身 的 体会 ， 他 了解 其中 的 主要矛盾 和 麻烦 ， 并 对

In [12]:
def multiclass_logloss(actual,predicted,eps=1e-15):
    if len(actual.shape) == 1:
        actual2 = np.zeros((actual.shape[0], predicted.shape[1]))
        for i, val in enumerate(actual):
            actual2[i, val] = 1
        actual = actual2

    clip = np.clip(predicted, eps, 1 - eps)
    rows = actual.shape[0]
    vsota = np.sum(actual * np.log(clip))
    return -1.0 / rows * vsota

In [13]:
lbl_enc = preprocessing.LabelEncoder()
y = lbl_enc.fit_transform(data.分类.values)

In [16]:

xtrain, xvalid, ytrain, yvalid = train_test_split(data.文本分词.values, y,
                                                  stratify=y,
                                                  random_state=42,
                                                  test_size=0.1, shuffle=True)
print (xtrain.shape)
print (xvalid.shape)

(8324,)
(925,)


In [25]:
def number_normalizer(tokens):
    """ 将所有数字标记映射为一个占位符（Placeholder）。对于许多实际应用场景来说，以数字开头的tokens不是很有用，    但这样tokens的存在也有一定相关性。通过将所有数字都表示成同一个符号，可以达到降维的目的。"""   
    return ("#NUMBER" if token[0].isdigit() else token for token in tokens)
class NumberNormalizingVectorizer(TfidfVectorizer):
    def build_tokenizer(self):
        tokenize = super(NumberNormalizingVectorizer,self).build_tokenizer()
        return lambda doc:list(number_normalizer(tokenize(doc)))

In [26]:
stwlist=[line.strip() for line in open('../../dataset/停用词库.txt',
'r',encoding='utf-8').readlines()]
tfv = NumberNormalizingVectorizer(min_df=3,
                                  max_df=0.5,
                                  max_features=None,
                                  ngram_range=(1, 2),
                                  use_idf=True,
                                  smooth_idf=True,
                                  stop_words = stwlist)

In [27]:
tfv.fit(list(xtrain) + list(xvalid))
xtrain_tfv =  tfv.transform(xtrain)
xvalid_tfv = tfv.transform(xvalid)

C:\Users\jianghaitao1\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:385: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['#NUMBER', 'lex', 'ｌｉ', 'ｚｘｆｉｔｌ'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


In [ ]:
clf = LogisticRegression(C=1.0,solver='lbfgs',multi_class='multinomial')
clf.fit(xtrain_tfv, ytrain)
predictions = clf.predict_proba(xvalid_tfv)

print ("logloss: %0.3f " % multiclass_logloss(yvalid, predictions))